In [1]:
%pip install spacy nltk
%spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%spacy` not found.


In [1]:


from datasets import load_from_disk, Dataset, concatenate_datasets, ClassLabel, Features, Value, Sequence
from transformers import BertTokenizer, BertForSequenceClassification, BasicTokenizer
from torch.utils.data import  DataLoader
from tqdm.notebook import tqdm
import torch
import nltk

import base


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [3]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [4]:
augmentatio_params = {"n_iter": 10, "p_mask":0.1, "p_pos": 0.3, "p_ng":0.2}

In [5]:
tokenizer = BasicTokenizer(do_lower_case=True)

In [6]:
train_data = load_from_disk("./data/sst2/train")
sentences = list(map(lambda e: e['sentence'], train_data))

In [7]:
pos_tag_word_map = {}


for s in sentences:
  for word, pos_tag in nltk.pos_tag(tokenizer.tokenize(s)):
    if pos_tag not in pos_tag_word_map.keys():
      pos_tag_word_map[pos_tag] = set()
    pos_tag_word_map[pos_tag].add(word)

    
pos_tag_word_map_list = {}
for pos_tag in pos_tag_word_map.keys():
  pos_tag_word_map_list[pos_tag] = []
  pos_tag_word_map_list[pos_tag] = list(pos_tag_word_map[pos_tag])

In [8]:
import random
iters = []
error_count = 0

for _ in range(augmentatio_params['n_iter']):
  data = {}
  for column in train_data.column_names:
    data[column] = []
  try:
    for row in train_data:
      res = []
      for word, pos_tag in nltk.pos_tag(tokenizer.tokenize(row['sentence'])):
        X = random.uniform(0,1)
        if X < augmentatio_params['p_mask']:
          res.append('[MASK]')
        elif X < augmentatio_params['p_pos']:
          res.append(random.choice(pos_tag_word_map_list[pos_tag]))
        else:
          res.append(word)
      if random.uniform(0,1) < augmentatio_params['p_ng']:
        n_gram_length = random.randint(20, 70)
        start = random.randrange(max(1, len(res)-n_gram_length))
        res = res[start: start+n_gram_length]
      synthetic_sample = ' '.join(res)
      data['sentence'].append(synthetic_sample)
      data['idx'].append(row["idx"])
      data['label'].append(row['label'])
    iters.append(data)
  except Exception as e:
    print(e)
    error_count += 1

print('\nMissed Entries', str(error_count))


Missed Entries 0


In [9]:
aug_datasets = []
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "label": ClassLabel(names=["negative", "positive"])
})

for iter in iters:
    new_train = Dataset.from_dict(iter)
    new_train = new_train.cast(ds_schema)
    aug_datasets.append(new_train)

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

In [10]:
print(aug_datasets[8][50])
print(train_data[50])

{'idx': 49478, 'sentence': "engrossing ' ' is the shapeless [MASK] move relying on the viewer to do most [MASK] the work .", 'label': 0}
{'idx': 49478, 'sentence': "solaris '' is a shapeless inconsequential move relying on the viewer to do most of the work . ", 'label': 0}


In [ ]:
tokenizer = BertTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2")
model = BertForSequenceClassification.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2", num_labels=2)
model.to(device)

torch.save(model.state_dict(), "./models/sst2/teacher.pth")

In [13]:
train_dataset = base.prepare_dataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
train_logits = base.generate_logits(train_dataloader, model, images=False)
train_dataset = train_dataset.add_column("logits", train_logits)
train_dataset = train_dataset.remove_columns(["token_type_ids", 'attention_mask'])
train_dataset.set_format(type='torch', columns=['logits', "labels"], device="cpu")

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

In [14]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


In [15]:
aug_clean_datasets = []
for dataset in tqdm(aug_datasets, total=(len(aug_datasets)), desc="Processing augmented datasets: "):
    aug_train_dataset = base.prepare_dataset(dataset, tokenizer)
    aug_train_dataloader = DataLoader(aug_train_dataset, batch_size=128, shuffle=False)
    aug_train_logits = base.generate_logits(aug_train_dataloader, model, images=False)
    aug_train_dataset = aug_train_dataset.add_column("logits", aug_train_logits)
    aug_train_dataset = aug_train_dataset.remove_columns(["token_type_ids", 'attention_mask'])
    aug_train_dataset.set_format(type='torch', columns=['logits', "labels"], device="cpu")

 
    #print(base.check_acc(aug_train_dataset, "Accuracy for given set BEFORE filtering: "))
    aug_train_dataset = base.remove_diff_pred_class(train_dataset, aug_train_dataset, pytorch_dataset=False)


    #print(base.check_acc(aug_train_dataset, "Accuracy for given set AFTER filtering: "))
    
    aug_train_dataset.reset_format()
    aug_clean_datasets.extend(aug_train_dataset)

Processing augmented datasets:   0%|          | 0/10 [00:00<?, ?it/s]

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

...

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tokenizing the provided dataset:   0%|          | 0/53879 [00:00<?, ? examples/s]

Generating logits for given dataset:   0%|          | 0/421 [00:00<?, ?it/s]

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

In [16]:
train_dataset.reset_format()

In [18]:
print("First iter check")
print(aug_clean_datasets[1997])
filtered_base = train_dataset.filter(lambda x: x['idx'] == aug_clean_datasets[1997]["idx"])
print(filtered_base[0])

print("Second iter check")
print(aug_clean_datasets[66699])
filtered_base = train_dataset.filter(lambda x: x['idx'] == aug_clean_datasets[66699]["idx"])
print(filtered_base[0])


First iter check
{'idx': 34853, 'sentence': 'plays like an motivated dialogue exercise worthwhile retard 101 .', 'labels': 0, 'input_ids': [101, 2399, 1176, 1126, 13241, 8556, 6730, 3869, 21053, 1231, 6817, 1181, 7393, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [3.193449020385742, -3.1790709495544434]}
{'idx': 34853, 'sentence': 'plays like an extended dialogue exercise in retard 101 . ', 'labels': 0, 'input_ids': [101, 2399, 1176, 1126, 2925, 8556, 6730, 1107, 1231, 6817, 1181, 7393, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [4.3499016761779785, -4.23635196685791]}
Second iter check
{'idx': 1817, 'sentence': 'each talky bore', 'labels': 0, 'input_ids': [101, 1296, 2037, 1183, 8475, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Filter:   0%|          | 0/53879 [00:00<?, ? examples/s]

{'idx': 1817, 'sentence': 'a talky bore ', 'labels': 0, 'input_ids': [101, 170, 2037, 1183, 8475, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'logits': [4.2185468673706055, -3.9582557678222656]}


In [20]:
print(train_dataset.features)

{'idx': Value(dtype='int32', id=None), 'sentence': Value(dtype='string', id=None), 'labels': ClassLabel(names=['negative', 'positive'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'logits': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}


In [21]:
ds_schema = Features({
    "idx": Value("int32"),
    "sentence": Value("string"),
    "labels": ClassLabel(names=["negative", "positive"]),
    "input_ids": Sequence(feature=Value(dtype='int32')),
    "logits": Sequence(feature=Value(dtype='float32')),
})

aug_dataset = Dataset.from_list(aug_clean_datasets)
aug_dataset = aug_dataset.cast(ds_schema)


Casting the dataset:   0%|          | 0/453735 [00:00<?, ? examples/s]

In [22]:
aug_dataset.set_format(type='torch', columns=['logits', "labels"], device="cpu")
train_dataset.set_format(type='torch', columns=['logits', "labels"], device="cpu")

In [23]:
print(base.check_acc(train_dataset, "Accuracy for base dataset: "))
print(base.check_acc(aug_dataset, "Accuracy for augmented dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/53879 [00:00<?, ?it/s]

Accuracy for base dataset:  0.9888268156424581


Calculating accuracy based on the saved logits:   0%|          | 0/453735 [00:00<?, ?it/s]

Accuracy for augmented dataset:  0.990148434658997


In [24]:
train_all_data = concatenate_datasets([train_dataset, aug_dataset])
train_all_data.set_format(type='torch', columns=['logits', "labels"], device="cpu")

In [25]:
print(base.check_acc(train_all_data, "Accuracy for combined dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/507614 [00:00<?, ?it/s]

Accuracy for combined dataset:  0.9900081558034254


In [26]:
print(train_all_data.column_names)

['idx', 'sentence', 'labels', 'input_ids', 'logits']


In [27]:
train_all_data.reset_format()
train_all_data = train_all_data.remove_columns(["idx", 'input_ids'])

In [28]:
train_all_data.save_to_disk("./data/sst2/train-logits-augmented")

Saving the dataset (0/1 shards):   0%|          | 0/507614 [00:00<?, ? examples/s]

In [29]:
train_dataset.reset_format()
train_dataset = train_dataset.remove_columns(["idx", 'input_ids'])
train_dataset.save_to_disk("./data/sst2/train-logits")

Saving the dataset (0/1 shards):   0%|          | 0/53879 [00:00<?, ? examples/s]

In [30]:
eval_data = load_from_disk("./data/sst2/eval")
eval_dataset = base.prepare_dataset(eval_data, tokenizer)
eval_dataloader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

In [31]:
test_data = load_from_disk("./data/sst2/test")
test_dataset = base.prepare_dataset(test_data, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [33]:
test_blank_data = load_from_disk("./data/sst2/test-blank")
test_blank_dataset = base.prepare_dataset(test_blank_data, tokenizer)
test_blank_dataloader = DataLoader(test_blank_dataset, batch_size=128, shuffle=False)

Tokenizing the provided dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [34]:
eval_logits = base.generate_logits(eval_dataloader, model, images=False)
test_logits = base.generate_logits(test_dataloader, model, images=False)
test_blank_logits = base.generate_logits(test_blank_dataloader, model, images=False)

Generating logits for given dataset:   0%|          | 0/7 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/106 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/15 [00:00<?, ?it/s]

In [35]:
eval_dataset.reset_format()
eval_dataset = eval_dataset.add_column("logits", eval_logits)
eval_dataset = eval_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [36]:
test_dataset.reset_format()
test_dataset = test_dataset.add_column("logits", test_logits)
test_dataset = test_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [37]:
test_blank_dataset.reset_format()
test_blank_dataset = test_blank_dataset.add_column("logits", test_blank_logits)
test_blank_dataset = test_blank_dataset.remove_columns(["idx", "token_type_ids", "input_ids", "attention_mask"])

In [38]:
eval_dataset.save_to_disk("./data/sst2/eval-logits")
test_dataset.save_to_disk("./data/sst2/test-logits")
test_blank_dataset.save_to_disk("./data/sst2/test-blank-logits")

Saving the dataset (0/1 shards):   0%|          | 0/872 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13470 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1821 [00:00<?, ? examples/s]

In [39]:
eval_data = load_from_disk("./data/sst2/eval-logits")
test_data = load_from_disk("./data/sst2/test-logits")

eval_data.set_format(type='torch', columns=['logits', "labels"], device="cpu")
test_data.set_format(type='torch', columns=['logits', "labels"], device="cpu")

print(base.check_acc(eval_data, "Accuracy for base eval dataset: "))
print(base.check_acc(test_data, "Accuracy for base test dataset: "))

Calculating accuracy based on the saved logits:   0%|          | 0/872 [00:00<?, ?it/s]

Accuracy for base eval dataset:  0.9231651376146789


Calculating accuracy based on the saved logits:   0%|          | 0/13470 [00:00<?, ?it/s]

Accuracy for base test dataset:  0.9896807720861173
